# Photometric detrending

EEG Meeting March 31, 2020

Brett Morris

## The signal(s) in the noise

Photometry from spacecraft are rarely "clean". The photometry often comes to the astronomer pre-reduced by the mission team with various trends embedded in the photometry which need to be removed before one can successfully measure the astrophysical signal of interest.

Here's a non-exhaustive list of the signals and noise that we expect in the CHEOPS photometry
1. Trends with time: spacecraft orbital phase, Sun angle, roll angle, temperature trends
2. Trends with stellar centroid position on detector: `x`, `y`, cross-terms (`x*y`)
3. Astrophysical signals of interest: transit, phase curve, secondary eclipse

## What gets delivered from CHEOPS

You get handed a bunch of FITS files that contain *record arrays*, which are matrices with labeled columns. The columns are things like: 
* times (BJD, UTC, etc.)
* roll angle
* stellar centroid position on detector
* contamination parameter
* **flux** and uncertainty

## What you already know

For a known transiting system, which make up the bulk of the CHEOPS schedule, you already know the planet-to-star radius ratio $R_p/R_\star$ and the mid-transit time $t_0$, so you can easily create a transit model which describes the expected astrophysical signal.


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def gaussian(theta, x): 
    """
    Gaussian signal
    """
    amp, x0, sigma = theta
    return amp * np.exp(-0.5 * (x0 - x)**2 / sigma**2)

## Create a set of simulated signal vectors

In [ ]:
np.random.seed(42)

# Set the number of data points
n = 1000

times = np.arange(n)

# Create position vectors for the centroid of the target
x = 10 * np.random.randn(n) + 500
y = 12 * np.random.randn(n) + 500

# Create sinusoidal varying background with time
background_period = 150
background = (100 * np.random.randn(n) + 1000 + 
              300 * np.sin(2 * np.pi / background_period * times) + 
              0.2 * times)

# Create astrophysical signal, which has a Gaussian shape
signal = gaussian([-500, 400, 50], times) + 50 * np.random.randn(n)

err = 50 * np.ones(n)

# Plot the vectors of the centroid positions, the time-varying 
# background, and the signal 
fig, ax = plt.subplots(1, 3, figsize=(10, 3))
ax[0].plot(x, y, 'k.')
ax[0].set(xlabel='X centroid', ylabel='Y centroid')
ax[1].plot(background, 'k.')
ax[1].set(xlabel='Time', ylabel='Background flux')
ax[2].plot(times, signal, 'k.')
ax[2].set(xlabel='Time', ylabel='Astrophysical Signal')
fig.tight_layout()

**Figure**: Several components of the measurements which we will use to detrend the photometry. The first plot shows the position of the stellar centroid on the detector (pixel coordinates). The second plot shows the time-varying background which is sinusoidal with a linear baseline trend. The third plot shows the astrophysical signal which we want to measure, which might be a transit-like signal with observational noise.

## Linear combination

The fundamental principle of this detrending technique is that the observed flux vector is a linear combination of vectors of other observed quantities. For example, 
$$ \mathrm{flux} = c_0 x + c_1 y + c_2 x y + c_3 \mathrm{(background)} + ... + c_n \mathrm{(signal)}$$. 

In the next cell, we'll explicitly define such a relationship by creating a matrix of the measurement vectors using the `np.vstack` command to stack the column vectors together into a matrix, called `X_true`. Then we'll randomly choose prefactor coefficients $\{c_i\}$ for each of the observed vectors which determine the weights of the vectors in the final `flux_observed` vector.

In [ ]:
np.random.seed(1984)

# Create a "hidden" relationship between each of the 
# input vectors (position, background, plus signal) 
# and the output vector (observed flux)

X_true = np.vstack([
    times - times.mean(),
    x - x.mean(), 
    y - y.mean(), 
    (x - x.mean()) * (y - y.mean()), 
    background, 
    signal
]).T

# Assign random weights to each of the vectors
betas_soln = np.random.random(size=X_true.shape[1])

flux_observed = X_true @ betas_soln

plt.plot(times, flux_observed, 'k.')
plt.gca().set(xlabel='Times', ylabel='Observed Flux')

**Figure**: The observed flux from the spacecraft will be a superposition of the measured vectors (and probably some that aren't measured, as well). The astrophysical signal at `t=400` is still visible, but it is clearly contaminated by the sinusoidal + linear trend background.

## Recovering the original signal with linear algebra

We can use [ordinary least squares](https://en.wikipedia.org/wiki/Ordinary_least_squares) to compute the weights of each of the measurement vectors which contribute to the observed signal.

In math-speak, the least squares estimator vector $\hat {\boldsymbol {\beta }}$ is given by

$${\displaystyle {\hat {\boldsymbol {\beta }}}=(\mathbf {X} ^{\rm {T}} N^{-1} \mathbf {X} )^{-1}\mathbf {X} ^{\rm {T}} N^{-1} \mathbf {y}, }$$

where $y$ is the observed flux vector, $X$ is the matrix of measurements to linearly combine into the observed flux vector, and $N$ is the diagonal matrix of flux uncertainties.

First, we'll define the linear regression algorithm, which solves for the weights directly using matrix inversion. Note, in Python >3.5 the `@` operator is the matrix multiplier.

In [ ]:
def linreg(X, flux, err):
    """
    Solve linear regression such that `X betas = flux`
    """
    # Make a matrix with `err` along the diagonal, square it, and take its inverse
    inv_N = np.linalg.inv(np.diag(err)**2)
    # Solve for betas 
    betas = np.linalg.inv(X.T @ inv_N @ X) @ X.T @ inv_N @ flux
    cov = np.linalg.inv(X.T @ inv_N @ X)
    return betas, cov

Next, we'll assemble our `X_obs` matrix of observation vectors, including the time, position vectors, a sinusoidal background trend, and the astrophysical signal model vector: 

In [ ]:
X_obs = np.vstack([
    # Trend in time
    times - times.mean(),
    
    # Positional terms
    x - x.mean(), 
    y - y.mean(),
    (x - x.mean()) * (y - y.mean()),
    
    # Background terms
    np.sin(2 * np.pi / background_period * times), 
    np.cos(2 * np.pi / background_period * times),
    np.ones(n),
    
    # Signal model
    gaussian([-500, 400, 50], times)
]).T

Now we'll run the linear regression routine on the matrix conditioned on the observed fluxes and their uncertainties. The best-fit model is the superposition of the vectors in `X_obs` multiplied by the weights that we found using the linear regression `betas`: 

In [ ]:
betas, cov = linreg(X_obs, flux_observed, err)

best_flux_model = X_obs @ betas

Let's plot the resulting "best flux model" vector (red) and the observed fluxes (black), and see how the residuals look:

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(6, 8), sharex=True)
ax[0].plot(times, flux_observed, 'k.', label='Observed')
ax[0].plot(times, best_flux_model, 'r', label='Model')
ax[0].legend(loc='lower right')
ax[0].set(ylabel='Flux')

ax[1].plot(times, flux_observed - best_flux_model, 'k.')
ax[1].set(xlabel='Time', ylabel='Residuals')
plt.show()

**Figure**: note that the residuals look like white noise -- it appears that we've included enough of the relevant vectors in the linear regression to remove most of the correlated noise.

Let's take a look at the light curve after it has been detrended against all of the vectors *except* the astrophysical signal vector: 

In [ ]:
plt.plot(times, signal, 'k.', label='Input signal')
plt.plot(times, flux_observed - (X_obs[:, :-1] @ betas[:-1]), 'r.', label='Recovered signal')
plt.legend(loc='lower right')
plt.xlabel('Time')
plt.ylabel('Flux')
plt.show()

**Figure**: note that the amplitude of the recovered signal is not exactly the same as the amplitude of the input signal – there is uncertainty associated with the matrix inversion step which propagates into uncertainty on the amplitude of the output signal.